In [12]:
from django_pandas.io import read_frame
import pandas as pd
import matplotlib
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr

import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'winesurvey.settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()

from survey.models import WineItem, SurveyQuestionResponse, SurveyABTestInstance

wine_data = pd.read_csv('data/wineitem_0519_0953.csv')
ab_data = pd.read_csv('data/surveyabtestinstance_0519_0953.csv')
survey_data = pd.read_csv('data/surveyquestionresponse_0519_0953.csv')

In [2]:
wine_data = wine_data[wine_data['price'] < 100]
valid_wines = wine_data['item key'].tolist()

In [5]:
from elosports.elo import Elo
eloLeague = Elo(k = 20, homefield=0)

for _, w in wine_data.iterrows():
    eloLeague.addPlayer(w['item key'])
    
for _, ab in ab_data.iterrows():
    if ab['winner_id'] in valid_wines and ab['loser_id'] in valid_wines:
        eloLeague.gameOver(winner=ab['winner_id'], loser=ab['loser_id'], winnerHome=False)

elo = pd.DataFrame(columns = ['item_key', 'elo'])
for team in eloLeague.ratingDict.keys():
    row = {'item_key': team, 'elo': eloLeague.ratingDict[team]}
    elo = elo.append(row, ignore_index=True)
elo.sort_values(by='elo')

elo = elo.join(wine_data.set_index('item key'), on='item_key')

In [ ]:
wrs.plot.scatter(x='price', y='elo')
lr = LinearRegression()
X = wrs['price'].to_numpy().reshape(-1, 1)
Y = wrs['elo'].to_numpy().reshape(-1, 1)
lr.fit(X,Y)
lr.coef_

In [16]:
elo.head(5)
#elo.groupby('varietal').mean()
#pearsonr(elo['price'], elo['elo'])

,item_key,elo,name,description,price,bottle image url,label type,closure type,country,region,...,vintage,point score,story,user rating,sale price,original url,tasting notes,winc product id,winc product code,wine product id
0,winc12387,1535.081013,2018 Porter & Plot® Pinot Noir,Porter & Plot showcases California's most exce...,41.99,https://d207gb2bfvg73.cloudfront.net/07191623/...,MI,CO,United States,Sonoma,...,2018.0,NaN,"By focusing on small-lot wines, Porter & Plot ...",4.3,34.99,NaN,NaN,12387.0,7191623.0,NaN
1,winc12366,1539.048166,2017 Yé-Yé Tempranillo,Our New World take on classic Spanish winemaki...,14.99,https://d207gb2bfvg73.cloudfront.net/06191603/...,MI,SC,Spain,Vino de Espana,...,2017.0,NaN,Originally a popular style of music in early 1...,3.9,12.99,NaN,NaN,12366.0,6191603.0,NaN
2,winc12343,1459.708785,2017 Baseline® Syrah,This cool-climate Syrah from winemaker Ryan Zo...,47.99,https://d207gb2bfvg73.cloudfront.net/15882073/...,MI,CO,United States,Santa Barbara County,...,2017.0,NaN,Winc winemaker Ryan Zotovich practices respect...,NaN,39.99,NaN,NaN,12343.0,15882073.0,NaN
3,winc12313,1498.904849,2018 Brethren of the Road® Riesling,"A light, dry California Riesling with bright f...",15.99,https://d207gb2bfvg73.cloudfront.net/02191550/...,WH,CO,United States,Monterey,...,2018.0,NaN,"Kenny Likitprakong, the winemaking partner who...",3.5,13.99,NaN,NaN,12313.0,2191550.0,NaN
4,winc12296,1568.880612,2017 L'Atelier du Sud® Red Blend,Evoking the charming beauty of the French coun...,15.99,https://d207gb2bfvg73.cloudfront.net/12181533/...,TR,CO,France,Costières de Nîmes,...,2017.0,NaN,Turn to this rustic blend of Grenache and Syra...,3.9,13.99,NaN,NaN,12296.0,12181533.0,NaN


In [9]:
qs = SurveyQuestionResponse.objects.all()
df = read_frame(qs)

df.pivot(index='survey_response',columns='survey_question',values='answer')


survey_question,age,bottles_per_month,buying_location[club_membership],buying_location[grocery],buying_location[online],buying_location[tasting_room],buying_location[wine_shop],email,format_likelihood[individual_bottles_site],format_likelihood[individual_bottles_winery],...,wine_knowledge,wine_love,wine_services[blueapron],wine_services[firstleaf],wine_services[localshop],wine_services[nakewines],wine_services[nytwineclub],wine_services[vinebox],wine_services[winc],wine_services[wineclub]
survey_response,,,,,,,,,,,,,,,,,,,,,
2020-05-01 22:47:13.349897+00:00 taylorsavage@gmail.com,2,5,2,0,1,2,2,taylorsavage@gmail.com,1,1,...,0,1,2,2,2,1,1,2,2,2
2020-05-06 01:42:51.008991+00:00 test@641.com,15,3,1,4,1,1,1,test@641.com,2,1,...,3,4,2,0,3,2,2,2,2,3
2020-05-06 04:06:19.168595+00:00 test@905.com,1,6,2,0,2,1,1,test@905.com,1,0,...,1,3,2,3,3,1,2,1,0,3
2020-05-17 17:37:39.928605+00:00 test@1036.com,23,56,2,1,1,1,1,test@1036.com,1,1,...,0,0,1,1,1,1,1,1,1,1
